In [1]:
import pandas  as pd

## 1. 파일준비및 전처리(결측치제거, 컬럼순서)

In [2]:
df = pd.read_csv("raw_data/10.5개시도_CCTV설치현황.csv")
df.head()

,address,purpose,cctv_cnt,set_yr,lat,lon
0,세종특별자치시 조치원읍 교리 129-1,다목적,3,NaN,36.605059,127.296286
1,세종특별자치시 조치원읍 남리144,다목적,2,NaN,36.597678,127.302452
2,세종특별자치시 조치원읍 남리 425,다목적,3,NaN,36.594863,127.300492
3,세종특별자치시 조치원읍 명리82,다목적,3,NaN,36.600620,127.301301
4,세종특별자치시 조치원읍 상리18-10,다목적,4,NaN,36.602434,127.303145


In [3]:
df.isnull().sum()

address      5218
purpose         0
cctv_cnt        0
set_yr      16810
lat             0
lon             0
dtype: int64

In [4]:
df = df.iloc[:,[0,1,2,5,4]].reset_index(drop=True)

In [5]:
df.head()

,address,purpose,cctv_cnt,lon,lat
0,세종특별자치시 조치원읍 교리 129-1,다목적,3,127.296286,36.605059
1,세종특별자치시 조치원읍 남리144,다목적,2,127.302452,36.597678
2,세종특별자치시 조치원읍 남리 425,다목적,3,127.300492,36.594863
3,세종특별자치시 조치원읍 명리82,다목적,3,127.301301,36.600620
4,세종특별자치시 조치원읍 상리18-10,다목적,4,127.303145,36.602434


In [17]:
# 추후 반복문의 replace를 위해 -> 반복문이 완료되면 누락되는 값이 있는데 이를 방지하기 위해
df["관할서"]="모름"

## 2. 해당 관할서 지정

In [7]:
# 위도경도 데이터 zip
location = list(zip(df["lon"], df["lat"]))

In [10]:
# geopandas, Point  라이브러리
import geopandas as gpd
from shapely.geometry import Point, Polygon

In [11]:
#위도 경도 데이터를 Point함수를 이용해서 변수로 만듦
#이를 리스트에 넣어줌
st_list = []
for i in location:
    test_code = Point(i)
    st_list.append(test_code)

In [12]:
#관할서 경계데이터 geopandas로 읽어오기
df_js = gpd.read_file('raw_data/1.5개시도_경찰서_관할경계.geojson', encoding='cp949')
df_js.head()

,NAME,PNAME,geometry
0,세종경찰서,충남청,"MULTIPOLYGON (((127.17202 36.73106, 127.17202 ..."
1,진주경찰서,경남청,"MULTIPOLYGON (((128.26697 35.12927, 128.26697 ..."
2,창원서부경찰서,경남청,"MULTIPOLYGON (((128.63363 35.22152, 128.63357 ..."
3,창원중부경찰서,경남청,"MULTIPOLYGON (((128.60966 35.15093, 128.60956 ..."
4,마산동부경찰서,경남청,"MULTIPOLYGON (((128.62696 35.21714, 128.62695 ..."


In [14]:
# 빠른 반복문 진행을 위해 변수지정
df_js_list = df_js["geometry"]
df_js_name = df_js["NAME"]

In [13]:
from tqdm import tqdm

In [18]:
# 위도경도 데이터가 관할경계데이터에 포함되면  관할서의 이름값을 df관할서의 "모름"을 관할서 이름으로 바꿔라 
# 넘 오래걸림...코드 디벨롶 요망

for z in tqdm(range(41)):
    for i in range(46203):
        if st_list[i].within(df_js_list[z]):
            df.iloc[i,-1] = df.iloc[i,-1].replace("모름",df_js_name[z])
            
        else:
            pass

100%|██████████| 41/41 [18:28<00:00, 27.04s/it]


In [20]:
#파일저장
df.to_csv("데이터별_관할서리스트/CCTV현황_관할서리스트")

In [ ]:
#강동경찰서는 데이터에 없으므로 추후에 결측치를 평균으로 대체해야 할 듯 하다

In [2]:
pd.read_csv("데이터별_관할서리스트/CCTV현황_관할서리스트", index_col=0)

,address,purpose,cctv_cnt,lon,lat,관할서
0,세종특별자치시 조치원읍 교리 129-1,다목적,3,127.296286,36.605060,세종경찰서
1,세종특별자치시 조치원읍 남리144,다목적,2,127.302452,36.597678,세종경찰서
2,세종특별자치시 조치원읍 남리 425,다목적,3,127.300492,36.594863,세종경찰서
3,세종특별자치시 조치원읍 명리82,다목적,3,127.301301,36.600620,세종경찰서
4,세종특별자치시 조치원읍 상리18-10,다목적,4,127.303145,36.602434,세종경찰서
...,...,...,...,...,...,...
46198,서울특별시 성북구 길음동 635-7,어린이보호,4,127.018663,37.613782,서울강북경찰서
46199,서울특별시 성북구 길음동 1280-4,어린이보호,4,127.015332,37.610174,서울성북경찰서
46200,서울특별시 성북구 길음동 1168,어린이보호,4,127.022334,37.609928,서울성북경찰서
46201,서울특별시 성북구 하월곡동 87-6,어린이보호,4,127.032080,37.609267,서울종암경찰서


-----
## 3. 파일 재 전처리

In [4]:
df = pd.read_csv("데이터별_관할서리스트/CCTV현황_관할서리스트", index_col=0)
df.head()

,address,purpose,cctv_cnt,lon,lat,관할서
0,세종특별자치시 조치원읍 교리 129-1,다목적,3,127.296286,36.605060,세종경찰서
1,세종특별자치시 조치원읍 남리144,다목적,2,127.302452,36.597678,세종경찰서
2,세종특별자치시 조치원읍 남리 425,다목적,3,127.300492,36.594863,세종경찰서
3,세종특별자치시 조치원읍 명리82,다목적,3,127.301301,36.600620,세종경찰서
4,세종특별자치시 조치원읍 상리18-10,다목적,4,127.303145,36.602434,세종경찰서


In [6]:
df.isnull().sum()

address     5218
purpose        0
cctv_cnt       0
lon            0
lat            0
관할서            0
dtype: int64

In [10]:
## address 결측치 "없음"으로 대체 ->주소따위 없어도됨 우리에겐 관할서가 있음

df = df.fillna("없음")

In [12]:
# 관할서 모름으로 되어있는 경우가 있음
df[df["관할서"].str.contains("모")]

,address,purpose,cctv_cnt,lon,lat,관할서
4759,경상남도 창원시 마산합포구 구산면 구복리 산153-1,기타,2,128.563892,35.059009,모름
15041,내곡동 (신원동) 69-7,생활방범,1,127.071160,37.430530,모름
35805,경기도 수원시 영통구 원천동 601-3,생활방범,1,127.065829,37.269854,모름
35806,경기도 수원시 영통구 원천동 601-9,생활방범,1,127.067959,37.269289,모름
38016,경기도 수원시 영통구 망포동 672-6,생활방범,4,127.045789,37.234150,모름
38167,경기도 수원시 영통구 망포동 234-3,교통정보수집,2,127.045751,37.234739,모름


In [ ]:
# 조사결과 아래와 같음
###마산중부경찰서
### 서초
### 수원남부경찰서
### 수원남부경찰서
### 수원남부경찰서
### 수원남부경찰서

In [18]:
#값 바꿔준다
df.iloc[4759,-1] = df.iloc[4759,-1].replace("모름","마산중부경찰서")
df.iloc[15041,-1] = df.iloc[15041,-1].replace("모름","서울서초경찰서")
df.iloc[35805,-1] = df.iloc[35805,-1].replace("모름","수원남부경찰서")
df.iloc[35806,-1] = df.iloc[35806,-1].replace("모름","수원남부경찰서")
df.iloc[38016,-1] = df.iloc[38016,-1].replace("모름","수원남부경찰서")
df.iloc[38167,-1] = df.iloc[38167,-1].replace("모름","수원남부경찰서")

In [21]:
#전처리 완료

## 4. 관할서별  CCTV개수 파악

+ 강동경찰서는 CCTV개수 누락됨 주의할것

In [52]:
# 관할서별 cctv개수 합계
df_1 = df.groupby("관할서", sort=True).sum("cctv_cnt")["cctv_cnt"]
df_cctv = pd.DataFrame(df_1).reset_index()
df_cctv

,관할서,cctv_cnt
0,마산동부경찰서,1218
1,마산중부경찰서,913
2,서울강남경찰서,3095
3,서울강북경찰서,2299
4,서울강서경찰서,2377
5,서울관악경찰서,4825
6,서울광진경찰서,6459
7,서울구로경찰서,3659
8,서울금천경찰서,4226
9,서울남대문경찰서,725


In [68]:
# 관할서명  수정
df_cctv["관할서"] = df_cctv["관할서"].str.replace("마산", "경남마산")                                     
df_cctv["관할서"] = df_cctv["관할서"].str.replace("창원", "경남창원")
df_cctv["관할서"] = df_cctv["관할서"].str.replace("수원", "경기수원")
df_cctv["관할서"] = df_cctv["관할서"].str.replace("진주", "경남진주")
df_cctv["관할서"] = df_cctv["관할서"].str.replace("진해", "경남진해")
df_cctv["관할서"] = df_cctv["관할서"].str.replace("세종", "충남세종")                                           

In [75]:
# 찾아보니 서울강동경찰서 cctv = 총 2619개 있음 ->강동구청 참고
# 데이터프레임에 추가
df_cctv.loc[40] = ["서울강동경찰서",2619]

In [82]:
#인덱스 정리
df_cctv = df_cctv.sort_values("관할서").reset_index(drop=True)

In [83]:
#전처리 완성본
df_cctv

,관할서,cctv_cnt
0,경기수원남부경찰서,4719
1,경기수원서부경찰서,2919
2,경기수원중부경찰서,4447
3,경남마산동부경찰서,1218
4,경남마산중부경찰서,913
5,경남진주경찰서,2709
6,경남진해경찰서,1142
7,경남창원서부경찰서,1050
8,경남창원중부경찰서,1429
9,서울강남경찰서,3095


In [85]:
#파일저장
df_cctv.to_csv("10_관할서별_cctv개수")